<a href="https://colab.research.google.com/github/Carol-Nunes/YOLO-custom-training/blob/main/C%C3%B3pia_de_YOLO_custom_training_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Montando o Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')


Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


# Clonando o Darknet

In [ ]:
!git clone 'https://github.com/AlexeyAB/darknet.git' '/content/gdrive/My Drive/yolo_custom_training/darknet'

fatal: destination path '/content/gdrive/My Drive/yolo_custom_training/darknet' already exists and is not an empty directory.


In [ ]:
# Compilar Darknet usando GPU: fazer alterações no makefile, GPU, CUDNN, OPENCV=1

%cd /content/gdrive/MyDrive/yolo_custom_training/darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!sed -i 's/CUDNN_HALF=0/CUDNN_HALF=1/' Makefile
!make

/content/gdrive/MyDrive/yolo_custom_training/darknet
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -DCUDNN_HALF -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -rdynamic -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -DCUDNN_HALF -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:945:23: warning: variable ‘rgb’ set but not used []8;;https://gcc.gnu.org/onlinedocs/gcc/Warning-Options.html#index-Wunused-but-set-variable-Wunused-but-set-variable]8;;]
  945 |                 float rgb[3];
      |                       ^~~
./src/image_opencv.cpp: In function ‘void cv_draw_object(image, float*, int, int, int*, float*, int*, int, char**)’:
./src/image_opencv.cpp:1443:14

# Atualizar o arquivo CFG da YOLO V3 para treinamento

In [ ]:
# Fazendo uma cópia do arquivo CFG para que seja possível fazer as alterações necessárias

!cp cfg/yolov3.cfg cfg/yolov3_custom_training.cfg

In [ ]:
# Fazendo as alterações necessárias

!sed -i 's/batch=1/batch=2/' cfg/yolov3_custom_training.cfg
!sed -i 's/subdivisions=1/subdivisions=8/' cfg/yolov3_custom_training.cfg
!sed -i 's/max_batches=500200/max_batches=2000/' cfg/yolov3_custom_training.cfg
!sed -i '610 s@classes=80@classes=1@' cfg/yolov3_custom_training.cfg
!sed -i '696 s@classes=80@classes=1@' cfg/yolov3_custom_training.cfg
!sed -i '783 s@classes=80@classes=1@' cfg/yolov3_custom_training.cfg
!sed -i '603 s@filters=255@filters=18@' cfg/yolov3_custom_training.cfg
!sed -i '689 s@filters=255@filters=18@' cfg/yolov3_custom_training.cfg
!sed -i '776 s@filters=255@filters=18@' cfg/yolov3_custom_training.cfg


In [ ]:
# Para treinamento é preciso comentar o tamanho e a subdivisão do lote de teste

!sed -i '3 s@batch=1@# batch=1@' cfg/yolov3_custom_training.cfg
!sed -i '4 s@subdivisions=1@# subdivisions=1@' cfg/yolov3_custom_training.cfg
!sed -i '6 s@# batch=64@batch=64@' cfg/yolov3_custom_training.cfg
!sed -i '7 s@# subdivisions=16@subdivisions=16@' cfg/yolov3_custom_training.cfg

# Configurar os arquivos de configuração de dados

In [ ]:
# Caminho para o diretório onde o arquivo será salvo
images_dir = '/content/gdrive/MyDrive/yolo_custom_training/butterfly'

# Lista de classes que você deseja adicionar ao arquivo 'classes.names'
classes = ['butterfly']

# Criar e escrever no arquivo 'classes.names'
with open(images_dir + '/classes.names', 'w') as file:
    for class_name in classes:
        file.write(class_name + '\n')

# Criando o arquivo path_data.data

In [ ]:
with open('/content/gdrive/MyDrive/yolo_custom_training' + '/path_data.data', 'w') as content:

    content.write('classes = 1\n')

    # Localização do arquivo train.txt
    content.write('train = ' + '/content/gdrive/MyDrive/yolo_custom_training/butterfly' + '/train.txt\n')

    # Localização do arquivo test.txt
    content.write('valid = ' + '/content/gdrive/MyDrive/yolo_custom_training/butterfly' + '/test.txt\n')

    # Localização do arquivo classes.names
    content.write('names = ' + '/content/gdrive/MyDrive/yolo_custom_training/butterfly' + '/classes.names\n')

    # Localizaçao da pasta de backup
    content.write('backup = backup')


In [ ]:
# Criação da pasta de backup

!mkdir /content/gdrive/MyDrive/yolo_custom_training/backup

mkdir: cannot create directory ‘/content/gdrive/MyDrive/yolo_custom_training/backup’: File exists


# Criando os arquivos train.txt e test.txt

In [ ]:
import os

all_images = [os.path.join('/content/gdrive/MyDrive/yolo_custom_training/butterfly/images_butterfly', img) for img in os.listdir('/content/gdrive/MyDrive/yolo_custom_training/butterfly/images_butterfly') if img.endswith('.png')]

# Definir o conjunto de teste e de treino (85% treino e 15% teste)

train_size = int(0.85 * len(all_images))

train_images = all_images[:train_size]
test_images = all_images[train_size:]

# Caminho completo das imagens e seus rótulos

train_images_paths = [os.path.join('/content/gdrive/MyDrive/yolo_custom_training/butterfly/images_butterfly', img) for img in train_images]
test_images_paths = [os.path.join('/content/gdrive/MyDrive/yolo_custom_training/butterfly/images_butterfly', img) for img in test_images]

train_labels_paths = [os.path.join('/content/gdrive/MyDrive/yolo_custom_training/butterfly/classes.txt', img.split('/')[-1].replace('.png', '.txt')) for img in train_images]
test_labels_paths = [os.path.join('/content/gdrive/MyDrive/yolo_custom_training/butterfly/classes.txt', img.split('/')[-1].replace('.png', '.txt')) for img in test_images]

# Criar arquivo train.txt

with open('/content/gdrive/MyDrive/yolo_custom_training/butterfly/train.txt', 'w') as f:
     for img_path, label_path in zip(train_images_paths, train_labels_paths):
            f.write(f"{img_path}\n")

# Criar arquivo test.txt

with open('/content/gdrive/MyDrive/yolo_custom_training/butterfly/test.txt', 'w') as f:
     for img_path, label_path in zip(test_images_paths, test_labels_paths):
            f.write(f"{img_path}\n")


# Cálculo dos pesos

In [ ]:
# Aqui será usado o Transfer Learning

!wget -P '/content/gdrive/MyDrive/yolo_custom_training/custom_weights' 'https://pjreddie.com/media/files/darknet53.conv.74'

--2025-02-10 23:40:14--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 162.0.215.52
Connecting to pjreddie.com (pjreddie.com)|162.0.215.52|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘/content/gdrive/MyDrive/yolo_custom_training/custom_weights/darknet53.conv.74.6’

darknet53.conv.74.6 100%[===================>] 154.96M  22.2MB/s    in 9.0s    

2025-02-10 23:40:23 (17.3 MB/s) - ‘/content/gdrive/MyDrive/yolo_custom_training/custom_weights/darknet53.conv.74.6’ saved [162482580/162482580]



# Início do Treinamento

In [ ]:
%cd /content/gdrive/MyDrive/yolo_custom_training/darknet
!chmod +x ./darknet

/content/gdrive/MyDrive/yolo_custom_training/darknet


In [ ]:
!./darknet detector train /content/gdrive/MyDrive/yolo_custom_training/path_data.data  /content/gdrive/MyDrive/yolo_custom_training/darknet/cfg/yolov3_custom_training.cfg /content/gdrive/MyDrive/yolo_custom_training/custom_weights/darknet53.conv.74 -dont_show

Streaming output truncated to the last 5000 lines.
Loaded: 0.000059 seconds
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.940731), count: 1, class_loss = 0.000003, iou_loss = 0.004038, total_loss = 0.004041 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 106 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
 total_bbox = 81649, rewritten_bbox = 0.000000 % 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.914486), count: 1, class_loss = 0.000008, iou_loss = 0.020757, total_loss = 0.020765 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 94 Avg (IOU: 0.000000), count: 1, class_loss = 0.000000, iou_loss = 0.000000, total_loss = 0.000000 
v3 (mse loss, Normalizer: (iou: 0